# LunarLander-v3 feat. Reinforcemente Learning 🚀

By
- Miguel González
- Javier Quesada

## Info

### Documentación

Problemas interesantes para Aprendizaje por refuerzo
 * Gymnasium: https://gymnasium.farama.org/environments/box2d/

### Instalación

%pip install gymnasium  
%pip install gymnasium[box2d] 

### Acciones adicionales

Pueden ser necesarias *antes* de instalar gymnasium[box2d].

#### En macos

pip uninstall swig  
xcode-select -—install (instala las herramientas de desarrollador si no se tienen ya)  
pip install swig  / sudo port install swig-python
pip install 'gymnasium[box2d]' # en zsh hay que poner las comillas  

#### en Windows

Si da error, se debe a la falta de la versión correcta de Microsoft C++ Build Tools, que es una dependencia de Box2D. Para solucionar este problema, puede seguir los siguientes pasos:
 * Descargar Microsoft C++ Build Tools desde https://visualstudio.microsoft.com/visual-cpp-build-tools/.
 * Dentro del instalador, seleccione la opción "Desarrollo para el escritorio con C++"
 * Reinicie su sesión en Jupyter Notebook o en Visual Studio.
 * Ejecute nuevamente el comando !pip install gymnasium[box2d] en la línea de comandos de su notebook.

#### En linux (colab)
  * pip install swig

## Dependencias 📦

In [ ]:
%pip install swig -q
%pip install gymnasium[box2d] -q
%pip install loky -q

In [1]:
import gymnasium as gym
import numpy as np
import pygame
import gymnasium.utils.play
from MLP import MLP
import copy
from loky import get_reusable_executor
import random
import matplotlib.pyplot as plt

## AutoPlay - humano 🕹️

In [ ]:
# prueba lunar lander por humano
env_human = gym.make("LunarLander-v3", render_mode="rgb_array")

lunar_lander_keys = {
    (pygame.K_UP,): 2,
    (pygame.K_LEFT,): 1,
    (pygame.K_RIGHT,): 3,
}
gymnasium.utils.play.play(env_human, zoom=1, keys_to_action=lunar_lander_keys, noop=0)

## Desarrollo de funciones 🧩

### Modelo 🧠

In [2]:
# construir modelo
LAYERS = [8, 6, 4]
model =  MLP(LAYERS)

### Política 🎯

In [3]:
def policy(observation, epsilon=0.1):
    if np.random.rand() < epsilon:  # 10% de las veces toma una acción aleatoria
        return np.random.randint(4)
    return np.argmax(model.forward(observation))  # 90% de las veces usa el modelo individuo MLP

### Entorno de exploración y función de ejecución 🌍

In [4]:
env = gym.make("LunarLander-v3")

def run ():
    """
    Esta función genera un escenario de juego y ejecuta la política definida en la función policy.
    Al acabar el juego (cuando se estrella o aterriza), devuelve la recompensa acumulada.
    """
    observation, info = env.reset() # se abre un escenario de juego
    racum = 0
    while True:
        action = policy(observation)
        observation, reward, terminated, truncated, info = env.step(action)
        racum += reward
        if terminated or truncated:
            return racum

### Funciones bioinspiradas 🧬

In [26]:
def sbx_crossover(parent1, parent2, eta_c=15):
    """
    Simulated Binary Crossover (SBX) para variables continuas en [0,1].   
    eta_c: distribution, controla la 'intensidad' del cruce.
    """
    
    # if np.random.rand() >= pc:
    #     return np.copy(parent1), np.copy(parent2)

    n_var = len(parent1)
    child1 = np.zeros(n_var)
    child2 = np.zeros(n_var)

    for i in range(n_var):
        x1 = parent1[i]
        x2 = parent2[i]

        if x1 > x2:
            x1, x2 = x2, x1

        if abs(x1 - x2) < 1e-14:
            # Si son prácticamente iguales, se copia
            child1[i] = x1
            child2[i] = x2
        else:
            # Beta determina el grado de cruce
            rand = np.random.rand()
            beta = 1.0 + (2.0 * (x1 - 0.0) / (x2 - x1))
            alpha = 2.0 - beta ** (-(eta_c + 1.0))
            if rand <= 1.0 / alpha:
                betaq = (rand * alpha) ** (1.0 / (eta_c + 1.0))
            else:
                betaq = (1.0 / (2.0 - rand * alpha)) ** (1.0 / (eta_c + 1.0))

            c1 = 0.5 * ((x1 + x2) - betaq * (x2 - x1))

            beta = 1.0 + (2.0 * (1.0 - x2) / (x2 - x1))
            alpha = 2.0 - beta ** (-(eta_c + 1.0))
            if rand <= 1.0 / alpha:
                betaq = (rand * alpha) ** (1.0 / (eta_c + 1.0))
            else:
                betaq = (1.0 / (2.0 - rand * alpha)) ** (1.0 / (eta_c + 1.0))

            c2 = 0.5 * ((x1 + x2) + betaq * (x2 - x1))

            c1 = min(max(c1, 0.0), 1.0)
            c2 = min(max(c2, 0.0), 1.0)

            # Aleatoriamente asignamos quién obtiene c1 y c2
            if np.random.rand() < 0.5:
                child1[i] = c2
                child2[i] = c1
            else:
                child1[i] = c1
                child2[i] = c2

    return child1, child2

In [40]:
# Función fitness
def evaluate_fitness(ind, num_eval_games):
    """Calcula el fitness promediando la recompensa de varias partidas."""
    total_reward = 0
    model.from_chromosome(ind)
    for _ in range(num_eval_games):
        total_reward += run()  # Ejecuta el agente en el entorno
    return total_reward/num_eval_games

def parallel_evaluation(population, num_eval_games, fitness_func):
    """Evalúa en paralelo el fitness de la población usando loky."""
    with get_reusable_executor() as executor:
        fitness_scores = list(executor.map(fitness_func, population, [num_eval_games]* len(population)))
    return fitness_scores

def sort_pop(population, fitness_scores):
    """Ordena la población según el fitness obtenido."""
    sorted_pop = sorted(zip(population, fitness_scores), key=lambda x: x[1], reverse=True)
    return [indiv for indiv, _ in sorted_pop], [fit for _, fit in sorted_pop]

# Función cruce
def pmx_crossover(parent1, parent2):
    """Cruce de mapeo parcial (PMX) entre dos cromosomas representados como listas de floats."""
    size = len(parent1)
    start, end = sorted(random.sample(range(size), 2))

    child1, child2 = parent1.copy(), parent2.copy()

    mapping1 = {parent1[i]: parent2[i] for i in range(start, end)}
    mapping2 = {parent2[i]: parent1[i] for i in range(start, end)}

    def fill_child(child, parent, mapping):
        for i in range(size):
            if child[i] == -1.0:
                gene = parent[i]
                while gene in mapping:
                    gene = mapping[gene]
                child[i] = gene
        return child

    child1[start:end], child2[start:end] = parent1[start:end], parent2[start:end]
    child1, child2 = fill_child(child1, parent2, mapping1), fill_child(child2, parent1, mapping2)

    return child1, child2

def crossover(ind1, ind2, pcross):
    if random.random() > pcross:
        return ind1.copy(), ind2.copy()
    return sbx_crossover(ind1, ind2)

# Función selección
def select(population, T: int) -> list[list]:
    """Return a copy of an individual by tournament selection. Population already ordered by fitness"""
    # print("dentro de select", len(population[0]))
    choices=random.sample(copy.copy(population), k=T)
    indices=[population.index(c) for c in choices]
    return population[np.argmin(indices)]

# Función mutación
def mutate(ind, pmut):
    """
    Aplica un operador de mutación seleccionado aleatoriamente a un individuo.
    """
    def mutate_uniform(ind, pmut):
        """
        Mutación uniforme: cada gen tiene probabilidad pmut de ser reemplazado
        por un valor aleatorio en [0,1].
        """
        for i in range(len(ind)):
            if np.random.rand() < pmut:
                ind[i] = np.random.rand()
        return ind

    def mutate_gaussian(ind, pmut, sigma=0.1):
        """
        Mutación gaussiana: con probabilidad pmut, se le suma
        una perturbación normal(0, sigma).
        """
        for i in range(len(ind)):
            if np.random.rand() < pmut:
                # pequeña perturbación
                perturb = np.random.normal(0, sigma)
                ind[i] += perturb
                # recortamos a [0,1]
                ind[i] = np.clip(ind[i], 0, 1)
        return ind
    mutation_operator = np.random.choice([mutate_gaussian])
    return mutation_operator(ind, pmut)

# Función evolutiva
def evolve(population, fit_func, num_eval_games=3, pmut=0.2, pcross=0.8, ngen=3000, T=6, trace=50, elitism=False):
    best_fitness = []
    worst_fitness = []
    mean_fitness = []
    for gen in range(ngen):

        # Evaluar fitness en paralelo
        fitness_scores = parallel_evaluation(population, num_eval_games, fit_func)

        # Ordenar por fitness
        sorted_population, sorted_scores = sort_pop(population, fitness_scores)
        
        # Guardar estadísticas
        best_fitness.append(sorted_scores[0])
        worst_fitness.append(sorted_scores[-1])
        mean_fitness.append(np.mean(sorted_scores))
        
        # Generar nueva población con cruce y mutación
        new_population = []
        if elitism:
            new_population.append(sorted_population[0].copy())
        while len(new_population) < len(population):
            parent1 = select(sorted_population[:len(population)//2], T)
            parent2 = select(sorted_population[:len(population)//2],T)
            childs = crossover(parent1, parent2, pcross=pcross)
            final_child1, final_child2 = mutate(childs[0], pmut=pmut), mutate(childs[1], pmut=pmut)

            # new_model1, new_model2 = MLP(LAYERS), MLP(LAYERS)
            # new_model1.from_chromosome(final_child1)
            # new_model2.from_chromosome(final_child2)

            new_population.append(final_child1 if type(final_child1) == list else final_child1.tolist())
            new_population.append(final_child2 if type(final_child2) == list else final_child2.tolist())


        population = [*new_population]
        if gen % trace == 0:
            print(f"Generation {gen} -> best fitness: {sorted_scores[0]}")
    print(f'Final best fitness: {sorted_scores[0]}')
    
    return population, best_fitness, worst_fitness, mean_fitness


## Neuroevolución 🔬🦾

In [41]:
POP_SIZE = 100
GENS = 3000
EVAL_GAMES = 3  # Número de partidas por individuo para calcular fitness

# Inicializar población aleatoria
population = np.random.uniform(-5, 5, size=(POP_SIZE, len(model.to_chromosome()))).tolist()

In [42]:
population, best_fitness, worst_fitness, mean_fitness = evolve(population,
                                                               evaluate_fitness,
                                                               num_eval_games=EVAL_GAMES,
                                                               ngen=GENS,
                                                               T=6,
                                                               trace=50,
                                                               elitism=False
                                                               )

Generation 0 -> best fitness: -89.21309339314479
Generation 50 -> best fitness: 7.486777941943316
Generation 100 -> best fitness: 17.90587074396812
Generation 150 -> best fitness: 30.72090756723662
Generation 200 -> best fitness: 8.99893949836717
Generation 250 -> best fitness: 11.23176746582567
Generation 300 -> best fitness: 65.60914893054031
Generation 350 -> best fitness: 79.67407163881704
Generation 400 -> best fitness: 6.6978360674809165
Generation 450 -> best fitness: -21.02847637262798
Generation 500 -> best fitness: -21.073970675170703
Generation 550 -> best fitness: 54.57413036443364
Generation 600 -> best fitness: -2.8856738602542626
Generation 650 -> best fitness: 126.89733305322096
Generation 700 -> best fitness: 39.065454474358525
Generation 750 -> best fitness: -2.2563433381499585
Generation 800 -> best fitness: -7.488949416553358
Generation 850 -> best fitness: 27.438253917139633
Generation 900 -> best fitness: 8.421061845878876
Generation 950 -> best fitness: -11.19893

KeyboardInterrupt: 

### Visualización de métricas de la evolución 📊📈

In [ ]:
plt.plot(best_fitness)
plt.plot(worst_fitness)
plt.plot(mean_fitness)
plt.xlabel('Generations')
plt.ylabel('Fitness')
plt.legend(['Best', 'Worst', 'Mean'])
plt.show()

## Prueba del mejor individuo 🏆🚀

In [ ]:
env_test = gym.make("LunarLander-v3", render_mode="human")

model_test = MLP(LAYERS)
model_test.from_chromosome(population[0])
def run_test():
    """
    Esta función genera un escenario de juego y ejecuta la política definida en la función policy.
    Al acabar el juego (cuando se estrella o aterriza), devuelve la recompensa acumulada.
    """
    observation, info = env_test.reset() # se abre un escenario de juego
    racum = 0
    while True:
        action = policy(observation)
        observation, reward, terminated, truncated, info = env_test.step(action)
        racum += reward
        if terminated or truncated:
            return racum
for e in range(10):
    run_test()

#### ¿No has tenido bastante?

Prueba a controlar el flappy bird https://github.com/markub3327/flappy-bird-gymnasium

pip install flappy-bird-gymnasium

import flappy_bird_gymnasium  
env = gym.make("FlappyBird-v0")

Estado (12 variables):
  * the last pipe's horizontal position
  * the last top pipe's vertical position
  * the last bottom pipe's vertical position
  * the next pipe's horizontal position
  * the next top pipe's vertical position
  * he next bottom pipe's vertical position
  * the next next pipe's horizontal position
  * the next next top pipe's vertical position
  * the next next bottom pipe's vertical position
  * player's vertical position
  * player's vertical velocity
  * player's rotation

  Acciones:
  * 0 -> no hacer nada
  * 1 -> volar